##### 7.4.1 침식 연산
침식 연산 : 객체를 침식시키는 연산

객체의 크기는 축소, 배경은 확장 -> 작은 크기의 객체를 제거(소금-후추 잡음, 임펄스 잡음 등)

입력 영상의 화소와 마스크 원소를 비교하여, 하나라도 일치하지 않으면 0(검은색)

- 돌출 부위는 여러 화소가 깎여 잡음이 완전히 제거됨




In [1]:
# 7.4.1 모폴로지 침식 연산
import numpy as np, cv2

# 침식 연산 함수
def erode(img, mask=None):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0] # 마스크 중심 좌표

    mcnt = cv2.countNonZero(mask)
    for i in range(ycenter, img.shape[0] - ycenter):
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1 # 마스크 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1 # 마스크 너비 범위
            roi = img[y1:y2, x1:x2] # 마스크 영역
            temp = cv2.bitwise_and(roi, mask)
            cnt = cv2.countNonZero(temp) # 일치 원소 개수 계산
            dst[i, j] = 255 if (cnt == mcnt) else 0 # 출력 화소에 저장
    return dst

image = cv2.imread("images_07/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data = [0, 1, 0,
        1, 1, 1,
        0, 1, 0]
mask = np.array(data, np.uint8).reshape(3,3)
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1] # 영상 이진화

dst1 = erode(th_img, mask)
dst2 = cv2.erode(th_img, mask) # 침식
# dst2 = cv2.morphologyEx(th_img, cv2.MORPH_ERODE, mask)

cv2.imshow("image", image)
cv2.imshow("binary image", th_img)
cv2.imshow("User erode", dst1)
cv2.imshow("CV erode", dst2)
cv2.waitKey(0)


-1

##### 7.4.2 팽창 영산
팽창 연산 : 객체를 팽창시키는 연산

객체의 크기는 확대되고, 배경은 축소됨, 객체 내부의 빈 공간이 메워짐

입력 화소와 팽창 마스크가 모두 불일치인 경우에만 0, 하나라도 일치하면 1


In [2]:
# 7.4.2 모폴로지 팽창 연산
import numpy as np, cv2

def dilate(img, mask):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0] # 마스크 중심 좌표

    for i in range(ycenter, img.shape[0] - ycenter):
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1 # 마스크 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1 # 마스크 너비 범위
            roi = img[y1:y2, x1:x2] # 마스크 영역
            temp = cv2.bitwise_and(roi, mask)
            cnt = cv2.countNonZero(temp) # 일치 원소 개수 계산
            dst[i, j] = 0 if (cnt == 0) else 255 # 출력 화소에 저장
    return dst

image = cv2.imread("images_07/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data = [0, 1, 0,
        1, 1, 1,
        0, 1, 0]
mask = np.array(data, np.uint8).reshape(3,3)
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1] # 영상 이진화

dst1 = dilate(th_img, mask)
dst2 = cv2.dilate(th_img, mask) # 팽창
# dst2 = cv2.morphologyEx(th_img, cv2.MORPH_DILATE, mask)

cv2.imshow("User dilate", dst1)
cv2.imshow("CV dilate", dst2)
cv2.waitKey(0)


-1

##### 7.4.3 열림 연산과 닫힘 연산
침식과 팽창은 따로만 쓰지는 않고 보통 함꼐 사용하는 경우가 많음

열림 연산(Opening) : 침식 연산 - 팽창 연산 (객체 축소 - 잡음 제거 - 객체 확대) -> 돌출 부분 돌아가지 않음

-> 노이즈가 없어짐

닫힘 연산(Closing) : 팽창 연산 - 침식 연산 (객체를 확대하면서 빈공간을 메운 후, 다시 축소)

-> 내부를 닫아줘서 닫힘이라고 외우면 편함

In [3]:
# 7.4.3 열림 연산과 닫힘 연산
import numpy as np, cv2
from Common.filters import erode, dilate

# 열림 연산 함수
def opening(img, mask):
    tmp = erode(img, mask) # 침식
    dst = dilate(tmp, mask) # 팽창
    return dst

# 닫힘 연산 함수
def closing(img, mask):
    tmp = dilate(img, mask) # 팽창
    dst = erode(tmp, mask) # 침식
    return dst

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data = [0, 1, 0,
        1, 1, 1,
        0, 1, 0]
mask = np.array(data, np.uint8).reshape(3,3)
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1] # 영상 이진화

dst1 = opening(th_img, mask)
dst2 = closing(th_img, mask)
dst3 = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, mask)
dst4 = cv2.morphologyEx(th_img, cv2.MORPH_CLOSE, mask, iterations=1)

cv2.imshow("User opening", dst1)
cv2.imshow("User closing", dst2)
cv2.imshow("CV opening", dst3)
cv2.imshow("CV closing", dst4)
cv2.waitKey(0)



-1